## Value overriding

#### Rationale

Overriding, or making exceptions, is an inherent inevitability in many data analysis applications. For example, when analyzing many similar data items, we can often devise a processing scheme that works well on most of the data items, yet requires some adjustments for outlier items (say, we may have thousands of images that are each normalized according to some default normalization scheme, but we need to make an exception for few of these images that were taken in different lighting conditions). *Quibbler* allows ways to override such default functional behavior in a simple, interactive, yet also transparent and well-documented fashion. Such excpetions are made by "overrding assignments" to functional quibs. 

#### The `allow_overriding` and `assigned_quibs` properties

When a _de novo_ assignment is being made to a specific quib (either programmatically, or via graphics-driven assignments) the assignment can be actualized as overriding of the focal quib, or can be propagated upstream, through inverse-assignments, to be actualized as overrides of upstream quibs. The choice of which quibs should be chosen for implementing the overriding assignment is controlled by the following two quib properties:

* `allow_overriding`: Specifies for each quib whether it accepts overriding assignments (`True` / `False`). By default, i-quibs allow overriding and f-quibs do not allow. In order to allow overriding of a specific f-quib, we need to explicitly set its `allow_overriding` to `True`.


* `assigned_quibs`: Indicates upstream quibs into which a _de novo_ assignment to the current quib should be inverse-assigned (see chapter on [inverse assignments]):

    * `None` (default): If there is only one option of an open upstream quib, inverse-assign to it. If multiple options exist, bring up a dialog box.

    * `{}`: Do not allow _de novo_ assignments to this quib.

    * `{quibs}`: Set of upstream quibs in which to actualize _de novo_ inverse assignments to the current quib. If multiple options exist, bring up a dialog box.

In the default settings, where `assigned_quibs=None` for all quibs and `allow_overriding=True` only for i-quibs, any _de novo_ assignment to a downstream f-quib is inverse-propagated all the way to the respective upstream input quibs, where it is ultimately actualized. 

Though, when overriding of specific intermediate f-quibs is enabled (`allow_overriding=True`), multiple options for actualizing a _de novo_ assignment to a downstream quib may be available. The choice among these options is determined by the `assigned_quibs` property of the quib to which the _de novo_ assignment is being made.

The following diagram depicts such inverse assignment behaviors:
[[/images/inverse_assignment_choice.png]]

#### Implementing overrding assignments

Consider the following simple example of defining a default value to be used for n items: 

In [1]:
# Imports
import pyquibbler as qb
from pyquibbler import iquib
qb.override_all()
import numpy as np

%matplotlib tk

In [2]:
# Number of data items:
n = iquib(8)

# Define default factor:
default_factor = iquib(np.array([7]))

# Define per-item factor by replicating the default factor 
# for each of the n items:
per_item_factor = np.tile(default_factor, n)
per_item_factor.get_value()

array([7, 7, 7, 7, 7, 7, 7, 7])

The `per_item_factor` is a function quib, providing a value to be used for processing each of our presumed data items. In this simple example, this value is simply a result of applying `tile` to the underlying i-quib `default_factor`. In general, though, such per-item decision could be a result of other, more complex, functions. Yet, as sophisticated as our automatic choice gets, we may still sometime need to make an exception to the default functional behavior. 

Say we want to override the functional behavior of the `per_item_factor` f-quib, substituting 9 in position 2. Since, by default, the f-quib does not allow overriding, when we assign:

In [3]:
per_item_factor[2] = 9

the assignment propagates upstream and actualized at the `default_factor` i-quib:

In [4]:
default_factor.get_value()

array([9])

This upstream assignment thereby changes the `per_item_factor` to 9 not only at position 2, but also in all other positions:

In [5]:
per_item_factor.get_value()

array([9, 9, 9, 9, 9, 9, 9, 9])

To allow overriding `per_item_factor` at the specified index, we need to set its `allow_overriding`:

In [6]:
per_item_factor.set_allow_overriding(True)

There are now two options for actualizing the assignment, either at the `per_item_factor` or at the upstream i-quib `default_factor`. Since we didn't specify which option to use (the `assigned_quibs` is `None` by default), if we now try to assign:

In [7]:
per_item_factor[2] = 12

we will get a dialog box indicating that there are two ways to fulfil our request, either by inverse assignment to the i-quib `default_factor`, or by overriding the functiuon quib `per_item_factor`:

[[/images/dialog_box_per_item_factor.png]]

Choosing to actualize at the `per_item_factor` will cause an overrding assignment to this functio quib:

In [8]:
per_item_factor.get_value()

array([ 9,  9, 12,  9,  9,  9,  9,  9])

As we see, the quib has been overridden to have a value of 12 at position 2. All other values remain functional: they are equal to the `default_factor`. Changing `default_factor` changes the downstream `per_item_factor` in all but the overridden positions:

In [9]:
default_factor[0] = 8
per_item_factor.get_value()

array([ 8,  8, 12,  8,  8,  8,  8,  8])

The choice we made in the dialog box is recorded in [MAOR: where?] of the quib:

So further assignments do not require bringing up the dialog box again:

In [10]:
per_item_factor[5] = 13
per_item_factor.get_value()

array([ 8,  8, 12,  8,  8, 13,  8,  8])

#### Assignments are actualized as a list of overrides to a quib's 'default' value

When we make assignments to a quib, these assignments are actualized as a list of overrides that apply to the quib's 'default' value. For i-quibs, this default value is the value they are assigned upon creation (stored as the quib's `args[0]`). For f-quibs, the default value is the output of their function. 

This override list is accessible through the `get_override_list()` method:

In [11]:
per_item_factor.get_override_list()

dict_values([Assignment(value=12, path=[PathComponent(indexed_cls=<class 'numpy.ndarray'>, component=2)]), Assignment(value=13, path=[PathComponent(indexed_cls=<class 'numpy.ndarray'>, component=5)])])

In addition, we can check which element positions are overridden, using the `get_override_mask()` method:

In [12]:
per_item_factor.get_override_mask().get_value() #MAOR: better syntax?

array([False, False,  True, False, False,  True, False, False])

#### Graphics-driven overriding assignments

Overriding assignments can also be used together with graphics-driven assignments, easily yielding interactive GUIs for default-overriding parameter specification. See [[quibdemo_default overriding]].

#### Clearing assignments by assigning the Default value
Overriding assignments at specific quib array subscripts can be cleared by assigning the `default` value:

In [ ]:
# TODO
per_item_factor[2] = qb.default
per_item_factor.get_value()

All assignments to a quib can be cleared using:

In [ ]:
# TODO
per_item_factor.assign_value(qb.default)

#### Out of range overriding are ignored

When we try to assign out of range, we get an exception. For example,

In [ ]:
per_item_factor[10] = 3
# yields: IndexError: index 10 is out of bounds for axis 0 with size 8

However, it is also possible that an originally legit assignment will become out of range. For example:

In [13]:
per_item_factor[7] = 3
per_item_factor.get_value()

array([ 8,  8, 12,  8,  8, 13,  8,  3])

This assignment will become out-of-range if we now change `n`. In this case, we get a warning, and the out-of-range assignment is ignored:

In [14]:
# TODO: no need for warning for out of bound
n.assign_value(7)
per_item_factor.get_value()

Attempted out of range assignment:
	data: [ 8  8 12  8  8 13  8]
	path: [PathComponent(indexed_cls=<class 'numpy.ndarray'>, component=7)]
	failed path component: 7
	exception: index 7 is out of bounds for axis 0 with size 7
Attempted out of range assignment:
	data: [ 8  8 12  8  8 13  8]
	path: [PathComponent(indexed_cls=<class 'numpy.ndarray'>, component=7)]
	failed path component: 7
	exception: index 7 is out of bounds for axis 0 with size 7


array([ 8,  8, 12,  8,  8, 13,  8])